<a href="https://colab.research.google.com/github/aawaskita/BukuPython/blob/master/_Classify_Clothing_Images_Usings_Neural_Networks_(MLP).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Classifying Images of Clothing


<table>
  <tr><td>
    <img src="https://cdn-images-1.medium.com/max/1200/1*az55lu4udkgtGdYX5inu2w.jpeg"
         alt="Fashion MNIST  Classification Problem"  width="800">
  </td></tr>
  <tr><td align="center">
     <a href="https://github.com/"></a><br/>
  </td></tr>
</table>

Welcome Back , in this tutorial, we'll build and train a machine learning model ( Unsupervised model) to classify images of clothing.

Our goal is to get the general sense of a ML project.

Let's Start 


## Install and import Packages

For this Colab, we'll need [TensorFlow Datasets](https://www.tensorflow.org/datasets/), an API which  we allow us to download and access the datasets we will work with.

We're also using a few helper packages like **Numpy**, **mathplotlib** and **math**

In [ ]:
!pip install -U tensorflow_datasets

In [ ]:
from __future__ import absolute_import, division, print_function


# TensorFlow and TensorFlow Datasets
import tensorflow as tf
import tensorflow_datasets as tfds
tf.logging.set_verbosity(tf.logging.ERROR)

# Another libraries
import math
import numpy as np
import matplotlib.pyplot as plt
import tqdm
import tqdm.auto
tqdm.tqdm = tqdm.auto.tqdm

print(tf.__version__)
tf.enable_eager_execution()  

## Import Our dataset : Fashion MNIST 

We work in this Colab with the [Fashion MNIST](https://github.com/zalandoresearch/fashion-mnist) dataset.

Similar to the MNIST digit dataset, the Fashion MNIST dataset includes:



1.   60,000 training examples
2.   10,000 testing examples
3.   10 classes
4.   28×28 grayscale/single channel images
  

 The images show individual articles of clothing at low resolution (28 $\times$ 28 pixels), as shown below:

<table>
  <tr><td>
    <img src="https://d2908q01vomqb2.cloudfront.net/f1f836cb4ea6efb2a0b1b99f41ad8b103eff4b59/2018/05/04/ImagesSageMaker3.png"
         alt="Fashion MNIST sprite"  width="400">
  </td></tr>
  <tr><td align="center">
     <a href="https://github.com/zalandoresearch/fashion-mnist"></a><br/>
  </td></tr>
</table>

We will use 60,000 images to train the network and 10,000 images to evaluate how accurately the network learned to classify images. 



In [ ]:
dataset, metadata = tfds.load('fashion_mnist', as_supervised=True, with_info=True)
train_dataset, test_dataset = dataset['train'], dataset['test']

Uploading the dataset returns the metadata, as well as a **training dataset** and a **test dataset** : 

* The model will be training using `train_dataset`.
* The model will be testing against `test_dataset`.

The images are 28 $\times$ 28 arrays, with pixel values is in the range `[0, 255]`. The *labels* are an array of integers, in the range `[0, 9]`. 
Each of these integers correspond to the *class* of clothing represented  by the image : 
The ten fashion class labels include:

1.   T-shirt/top
2.   Trouser/pants
3.    Pullover shirt
4.    Dress
5.   Coat
6.   Sandal
7.   Shirt
8.   Sneaker
9.   Bag
10.  Ankle boot

<table>
  <tr><td>
    <img src="https://cdn-images-1.medium.com/max/1600/1*-kpgaee9X9Gm-SrQKdk_og.png"
         alt="Fashion MNIST sprite"  width="410">
  </td></tr>
  <tr><td align="center">
     <a href="https://github.com/zalandoresearch/fashion-mnist"></a><br/>
  </td></tr>
</table>

Each image is mapped to a single label. 

Since **class names** are not included in the dataset, it should be stored here to be used when tracing images:

In [ ]:
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 
               'Sandal',      'Shirt',   'Sneaker',  'Bag',   'Ankle boot']

### Explore our dataset

Let's explore the format of the dataset before training the model. 

The following code shows that there are 60,000 images in the training set, and 10000 images in the test set. 


In [ ]:
num_train_examples = metadata.splits['train'].num_examples
num_test_examples = metadata.splits['test'].num_examples
print("Number of training examples: {}".format(num_train_examples))
print("Number of test examples:     {}".format(num_test_examples))

## Preprocess the data

The value of each pixel in the image data is an integer in the range `[0,255]`. But for to work properly,we need to be normalize  these values  to the range `[0,1]`. So for that we should create a simple normalization function, and then apply it to each image in the test and train datasets.

In [ ]:
def normalize(images, labels):
  images = tf.cast(images, tf.float32)
  images /= 255
  return images, labels

train_dataset =  train_dataset.map(normalize)
test_dataset  =  test_dataset.map(normalize)






Let's plot an image to see what it looks like.

In [ ]:
for image, label in test_dataset.take(1):
  break
image = image.numpy().reshape((28,28))

plt.figure()
plt.imshow(image, cmap=plt.cm.binary)
plt.colorbar()
plt.grid(False)
plt.show()

Now we display the first 25 images from the *training set* and the class name below each image. 

Now, we're ready to build and train the network.

In [ ]:
plt.figure(figsize=(10,10))
i = 0
for (image, label) in test_dataset.take(25):
    image = image.numpy().reshape((28,28))
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(image, cmap=plt.cm.binary)
    plt.xlabel(class_names[label])
    i += 1
plt.show()

## Build the model

As we saw in the previous tutorial, building the machine learning model requires configuring the layers of the model and then compiling it.

We will build a 2-layer feedforward neural network with 128 units in the hidden layer. the hidden layer will compute a linear function which is then passed into a ReLU activation function. 

Finally, we will use a Softmax function on the output from our network, to create 10 outputs (1 output for each target class).


<table>
  <tr><td>
    <img src="https://harishnarayanan.org/images/writing/artistic-style-transfer/neural-network-1-hidden.svg"
         alt="Fashion MNIST sprite"  width="410">
  </td></tr>
  <tr><td align="center">
     <a href="https://github.com/zalandoresearch/fashion-mnist"></a><br/>
  </td></tr>
</table>


Let's setup the layers.

### Setup the layers

The core element of a neural network is the *layer*. A layer help us to extracts a representation of the input data. A series of connected layers gives a meaningful representation for the problem to be solved.

**Note :** Lots of deep learning consists of connecting simple layers. Most layers, such as `tf.keras.layers.Dense`, contain internal parameters that are learned  during training.

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28, 1)),
    tf.keras.layers.Dense(128, activation=tf.nn.relu),
    tf.keras.layers.Dense(10,  activation=tf.nn.softmax)
])

Our model has three layers:

* **input** `tf.keras.layers.Flatten`  :  This layer will transform the images from a 2d-array of 28 $\times$ 28 pixels), to a 1d-array (lists) of 784 pixels (28\*28). This layer has no parameters to learn, as it only reformats the data.

* **"hidden"** `tf.keras.layers.Dense` : A densely connected layer of 128 neurons. The principe is very simple : Each neuron takes as input the 784 nodes of the previous layer, weighting these inputs according to hidden parameters (**weights** )that will be learned during the training, then sends a unique value to the next layer.

* **output** `tf.keras.layers.Dense` : A 10-neuron **softmax** ( [Activation Function](https://towardsdatascience.com/activation-functions-neural-networks-1cbd9f8d91d6) )layer. Each neuron represents a class of clothing. As in the previous layer, each neuron takes input from the 128 nodes in the layer before it, weights thats input according to learned parameters, and outputs a value in the range `[0, 1]`, representing the probability that the image belongs to that class. The sum of all 10 neuron values is 1.


### Compile the model

Now our model is ready for training but it needs a few more settings. These are added during the compiling step:


* *Loss function* :  An algorithm for measuring how far the model's outputs are from the desired output. 
* *Optimizer* : An algorithm to adjust the internal parameters of the model to minimize  loss.
* *Metrics* : Used to monitor the training and testing steps. OUr example example uses *accuracy*, the fraction of correctly classified images.

In [ ]:
model.compile(optimizer='adam', 
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

## Train the model

It’s time to pass our Fashion MNIST data into our neural network and see how it performs!

First, we define the iteration behavior for the train dataset:
1. Repeat forever by specifying `dataset.repeat()` 

2. The `dataset.shuffle(60000)` randomizes the order so our model cannot learn anything from the order of the examples.

3. And `dataset.batch(32)` tells `model.fit` to use batches of 32 images and labels when updating the model variables.

the training step is performed by calling the `model.fit` method : 

1. `train_dataset`:  used to feed the training data to the model 

2. The model learns to associate images and labels.

3. The `epochs=5` parameter limits training to 5 full iterations of the training dataset ( total of 5 * 60000 = 300000 examples) .



In [ ]:
BATCH_SIZE = 32
train_dataset = train_dataset.repeat().shuffle(num_train_examples).batch(BATCH_SIZE)
test_dataset = test_dataset.batch(BATCH_SIZE)

In [ ]:
model.fit(train_dataset, epochs=5, steps_per_epoch=math.ceil(num_train_examples/BATCH_SIZE))

As the model trains, the loss and accuracy metrics are displayed. 

## Evaluate accuracy

Then we will compare the performance of the model on the test dataset. In other words, we will evaluate the accuracy. So, for that, we will use all the examples of the test dataset.

In [ ]:
test_loss, test_accuracy = model.evaluate(test_dataset, steps=math.ceil(num_test_examples/32))
print('Accuracy on test dataset:', test_accuracy)

**Note : ** It turns out that the accuracy of the test dataset is less than that of the training dataset. This is perfectly normal because the model was formed on `train_dataset` : When the model sees images (from `test_dataset`), it has never seen during training . We can expect a decrease in performance.

## Make predictions and explore

Now, We can use  the model trained to make predictions about some images.

In [ ]:
for test_images, test_labels in test_dataset.take(1):
  test_images = test_images.numpy()
  test_labels = test_labels.numpy()
  predictions = model.predict(test_images)

In [ ]:
predictions.shape


Here, the model has predicted the label for each image in the testing set. Let's take a look at the first prediction for example:

In [ ]:
predictions[0]

As we are 10 output, the our prediction is an array of 10 numbers. These describe the "confidence" of the model that the image corresponds to each of the 10 different clothes. We can see which label has the highest confidence value:

In [ ]:
np.argmax(predictions[0])

So the model is most confident that this image is a shirt, or `class_names[6]`. And we can check the test label to see this is correct:

In [ ]:
test_labels[0]

**Thanks for your attention **

I wish that you enjoyed this Notebook. In the next one, we'll see how to calssify clothing images **EASILY**  using the Convolution Neural Networks (CNN). 

<table>
  <tr><td>
    <img src="https://previews.123rf.com/images/123vector/123vector1409/123vector140900073/31489018-vector-illustration-of-see-you-soon-yellow-note-on-white-background.jpg"
         alt="Fashion MNIST sprite"  width="200">
  </td></tr>
  <tr><td align="center">
     <a href="https://github.com/zalandoresearch/fashion-mnist"></a><br/>
  </td></tr>
</table>

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.linkedin.com/in/thamer-saraei-472300124/"><img src="http://icons.iconarchive.com/icons/limav/flat-gradient-social/32/Linkedin-icon.png" />Join me </a>
  </td>
  <td>
    <a target="_blank" href="https://www.youtube.com/channel/UC8Dt8pO_EqhP9unfnMd-64A?view_as=subscriber"><img src="http://icons.iconarchive.com/icons/emey87/social-button/32/youtube-icon.png" />Join me </a>
  </td>
  <td>
    <a target="_blank" href="https://www.facebook.com/timopyr/"><img src="https://icon-icons.com/icons2/1269/PNG/32/1497553311-103_84832.png" />Join me</a>
  </td>
</table>